In [52]:
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from pyxirr import xirr
pd.options.display.float_format = '{:,.2f}'.format


# Imputs

In [53]:
df_input=pd.read_csv('./input_fluxo_caixa.csv',header=None,index_col=0)
df_input=df_input.rename(columns={1:'valor_variavel'})

In [54]:
valor_imovel=df_input.loc['valor_imovel','valor_variavel']
entrada_pp=df_input.loc['entrada_pp','valor_variavel']
n_parcelas=int(df_input.loc['n_parcelas','valor_variavel'])                         # 35 anos
juros_nominal=df_input.loc['juros_nominal','valor_variavel']                        # a.a
area=df_input.loc['area','valor_variavel']

# Custos Iniciais

In [55]:
valor_entrada=valor_imovel*entrada_pp

custo_itbi=0.03*valor_imovel
custo_escritura=df_input.loc['custo_escritura','valor_variavel']
custo_registro=df_input.loc['custo_registro','valor_variavel']
custo_avaliacao=df_input.loc['custo_avaliacao','valor_variavel']

total_custos_iniciais=valor_entrada+custo_itbi+custo_escritura+custo_registro+custo_avaliacao


# Cálculo financiamento

In [56]:
juros_efetivo=((1+juros_nominal/12)**(12)-1)            # Efeito anual dos juros sobre juros mensais.  

valor_finaciado=valor_imovel-valor_entrada

encargos_finaciamento=df_input.loc['encargos_finaciamento','valor_variavel']

In [57]:
amortizacao=valor_finaciado/n_parcelas
saldo_devedor=valor_finaciado

# Cálculo do juros em função do mês
def parcela(n):
    saldo_devedor=valor_finaciado-(n-1)*amortizacao
    juros=saldo_devedor*juros_nominal/12
    return amortizacao+juros+encargos_finaciamento

df=pd.DataFrame(columns=['parcela','saldo_devedor'],index=range(0,n_parcelas))


for i in range(0,n_parcelas+1):
    if i==0:
        df.loc[i,'saldo_devedor']=valor_finaciado-i*amortizacao
        df.loc[i,'parcela']=0
    else:

        df.loc[i,'saldo_devedor']=valor_finaciado-i*amortizacao
        df.loc[i,'parcela']=parcela(i)

# Custos fixos

In [58]:
condominio=df_input.loc['condominio','valor_variavel']
iptu_am=df_input.loc['iptu_aa','valor_variavel']/12
luz=df_input.loc['luz','valor_variavel']

total_custos_fixos=condominio+iptu_am+luz

In [59]:
for i in range(0,len(df)):
    df.loc[i,'custos_fixos']=total_custos_fixos
    if i==0:
        df.loc[i,'custos_iniciais']=total_custos_iniciais
    else:
        df.loc[i,'custos_iniciais']=0

# Custos com reforma

In [60]:
obra=df_input.loc['obra','valor_variavel']
prazo_obra=df_input.loc['prazo_obra','valor_variavel']                # Prazo da obra em mesês

def reforma(i, n,total):
    if i==0:
        return 0
    elif i<=n:
        return total/n
    else:
        return 0

for i in range(0,len(df)):
    df.loc[i,'reforma']=reforma(i,prazo_obra,obra)

# Cenários de venda

In [61]:
valor_venda=df_input.loc['valor_venda','valor_variavel']
prazo_venda=int(prazo_obra+df_input.loc['prazo_venda_pos_obra','valor_variavel'])                # Prazo da obra em mesês + venda
comissao=df_input.loc['comissao','valor_variavel']

In [62]:
def venda(i, n,total):
    if i==n:
        print(f'Comissão: {comissao*valor_venda}')
        return -total*(1-comissao)
    else:
        return 0

for i in range(0,len(df)):
    df.loc[i,'venda']=venda(i,prazo_venda,valor_venda)
df['total']=df.sum(axis=1)
df['total']=df['total']-df['saldo_devedor']

# quitação do saldo devedor e anulação das despesas à partir da venda
for i in range(0,len(df)):
    if i>prazo_venda:
        df.loc[i,'total']=0
    elif i==prazo_venda:
        df.loc[i,'total']=df.loc[i,'total']+df.loc[i,'saldo_devedor']

Comissão: 16000.0


In [63]:
delta = relativedelta(months=1)
delta_d = timedelta(days=1)
start_date = datetime.today().date()+15*delta_d

for i in range(0, len(df)):
    if i==0:
        df.loc[i,'data']=start_date
    else:
        df.loc[i,'data']=start_date+i*delta

df=df[df['total']!=0]
df.to_csv('output.csv', header=True, float_format='%.2f')
df.head(12)

,parcela,saldo_devedor,custos_fixos,custos_iniciais,reforma,venda,total,data
0,0,"335,000.00",721.67,"364,100.00",0.00,0.00,"364,821.67",2022-08-18
1,"3,158.16","334,202.38",721.67,0.00,"15,000.00",0.00,"18,879.83",2022-09-18
2,"3,152.85","333,404.76",721.67,0.00,"15,000.00",0.00,"18,874.52",2022-10-18
3,"3,147.54","332,607.14",721.67,0.00,"15,000.00",0.00,"18,869.21",2022-11-18
4,"3,142.23","331,809.52",721.67,0.00,0.00,0.00,"3,863.89",2022-12-18
5,"3,136.92","331,011.90",721.67,0.00,0.00,0.00,"3,858.58",2023-01-18
6,"3,131.61","330,214.29",721.67,0.00,0.00,"-784,000.00","-449,932.44",2023-02-18


In [64]:
# calculo da IRR
def cal_irr(x):
    
    dates=list(x['data'])
    amounts=list(x['total'])
    t_aa=xirr(dates, amounts)
    t_am=(1+t_aa)**(1/12)-1
    print('TII a.a.(%): ',t_aa*100)
    print('TII a.m.(%): ',t_am*100)
    print('Lucro: ',-x['total'].sum())
    # return (t_aa*100,t_am*100,-x['total'].sum())

In [65]:
cal_irr(df)

TII a.a.(%):  10.448163023331162
TII a.m.(%):  0.8315727836996079
Lucro:  20764.745535714086


In [66]:
valor_m2_aquisicao=valor_imovel/area
print(f'm2 aquisição: {valor_m2_aquisicao}')
valor_m2_venda=valor_venda/area
print(f'm2 reforma: {obra/area}')
print(f'm2 venda: {valor_m2_venda}')


m2 aquisição: 10151.515151515152
m2 reforma: 681.8181818181819
m2 venda: 12121.212121212122


In [67]:
11*66+70

796

In [68]:
### projeções de inflação e crescimento econômico para evolução dos preços de venda